# Introduction

### This notebook aims to detect anomalies in financial markets using the XGBoost model.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from flask import Flask, request, jsonify
import threading
import requests

# Data Cleaning

In [ ]:
dataset = pd.read_excel('../data/FinancialMarketData.xlsx', sheet_name='EWS' )

# moving averages   
dataset['VIX_moving_average'] = dataset['VIX'].rolling(window=7).mean()
dataset['BDIY_moving_average'] = dataset['BDIY'].rolling(window=7).mean()
dataset['CRY_moving_average'] = dataset['CRY'].rolling(window=7).mean()
dataset['Cl1_moving_average'] = dataset['Cl1'].rolling(window=7).mean()
dataset['USGG2YR_moving_average'] = dataset['USGG2YR'].rolling(window=7).mean()
dataset['GT10_moving_average'] = dataset['GT10'].rolling(window=7).mean()
dataset['DXY_moving_average'] = dataset['DXY'].rolling(window=7).mean()

# Ratios
dataset['DXY_to_VIX'] = dataset['DXY'] / dataset['VIX']

# Differences
dataset['VIX_diff'] = dataset['VIX'].diff()

dataset['Year'] = dataset['Data'].dt.year
dataset['Month'] = dataset['Data'].dt.month
dataset['Day'] = dataset['Data'].dt.day
dataset['DayOfWeek'] = dataset['Data'].dt.dayofweek

#correlation
correlation_matrix = dataset.corr()
low_cor_cols=correlation_matrix['Y'][abs(correlation_matrix['Y'])<0.04].index

#cleaning data
dataset_cleaned=dataset.drop(columns=low_cor_cols)

#droping Date Column
dataset_cleaned = dataset_cleaned.drop(columns=['Data'])

#dataset_cleaned.describe()


# Data Preperation

In [ ]:
X= dataset_cleaned.drop(columns=['Y'])
y=dataset_cleaned['Y']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Model Training

## Handling Class Imbalance

In [ ]:
class_counts = Counter(y_train)
negatives = class_counts[0]  # Normal class
positives = class_counts[1]  # Anomaly class

# Scale_pos_weight calcul
scale_pos_weight = negatives / positives
print(f"Scale Pos Weight: {scale_pos_weight}")


## Training the XGBoost Classifier

In [ ]:
# Model inisialization
model = XGBClassifier(
    n_estimators=131, 
    learning_rate=0.3, 
    random_state=43, 
    scale_pos_weight=scale_pos_weight
)

model.fit(X_train, y_train)

#  prediction model
y_pred = model.predict(X_test)

print("Classification Report :")
print(classification_report(y_test, y_pred))


# Investment Strategy Implementation

### In this section, we will define and implement a data-driven investment strategy based on the model's predictions.


In [ ]:
dataset_cleaned['Anomaly'] = model.predict_proba(X)[:, 1] >= 0.1  
dataset_cleaned['price'] = dataset_cleaned['VIX']

position = 0  # Initial position (no active trade)
entry_price = None 

def define_strategy(row):
    global entry_price, position  # Using global variables to maintain state

    # Buy if conditions are met
    if row['Anomaly'] and row['VIX'] < row['VIX_moving_average']:
        if position == 0:  # Buy only if there is no active position
            entry_price = row['price']
            position = 1  # Active position
            return 'Buy'
    
    # Sell if conditions are met
    elif position > 0 and (row['price'] > entry_price * 1.1 or row['price'] < entry_price * 0.95):
        position = 0  # Close the position
        entry_price = None  # Reset entry price
        return 'Sell'
    
    # Do not trade on weekends
    elif row['DayOfWeek'] in [5, 6]:  # Assuming weekends are Saturday and Sunday
        return 'Hold'

    # Default action: hold
    return 'Hold'

# Application de la stratégie
dataset_cleaned['Strategy'] = dataset_cleaned.apply(define_strategy, axis=1)


## Backtesting

In [ ]:
# Initialization
initial_portfolio = 100000  # Initial capital
portfolio = initial_portfolio
position = 0  # Current position (amount of asset held)
entry_price = None  # Entry price for an active position
cash = initial_portfolio  # Uninvested cash amount
transaction_fee = 0.001  # Transaction fee (0.1% per transaction)

portfolio_values = []  # List to store portfolio values at each step
transactions = []  # List to log transactions

# Strategy simulation
for index, row in dataset_cleaned.iterrows():
    # Buy if the strategy indicates 'Buy' and no active position
    if row['Strategy'] == 'Buy' and position == 0:
        entry_price = row['price'] * (1 + transaction_fee)  # Include buy transaction fee
        position = cash / entry_price  # Calculate the amount bought
        cash = 0  # All cash is invested
        transactions.append(f"Buy at {row['price']} (adjusted: {entry_price}) on index {index}")
    
    # Sell if the strategy indicates 'Sell' and there is an active position
    elif row['Strategy'] == 'Sell' and position > 0:
        sell_price = row['price'] * (1 - transaction_fee)  # Include sell transaction fee
        cash = position * sell_price  # Calculate cash gained
        portfolio = cash  # Update portfolio value
        position = 0  # Reset the position
        transactions.append(f"Sell at {row['price']} (adjusted: {sell_price}) on index {index}")
    
    # Append the current portfolio value
    current_portfolio_value = cash + (position * row['price'] if position > 0 else 0)
    portfolio_values.append(current_portfolio_value)

# Calculate total return
total_return = (portfolio_values[-1] - initial_portfolio) / initial_portfolio * 100
print(f"Total Return: {total_return:.2f}%")

# Add the Portfolio_Value column to the dataset
dataset_cleaned['Portfolio_Value'] = portfolio_values

# Print the final portfolio value
print(f"Final Portfolio Value: {portfolio_values[-1]}")

# Print the transaction log
print("\nTransactions:")
for transaction in transactions:
    print(transaction)




##  Visualization Of The Portfolio

In [ ]:
buy_points = dataset_cleaned[dataset_cleaned['Strategy'] == 'Buy'].index
sell_points = dataset_cleaned[dataset_cleaned['Strategy'] == 'Sell'].index

plt.figure(figsize=(20, 10))
plt.plot(dataset_cleaned['Portfolio_Value'], label="Portfolio Value")
plt.scatter(buy_points, dataset_cleaned.loc[buy_points, 'Portfolio_Value'], color='green', label='Buy', marker='^')
plt.scatter(sell_points, dataset_cleaned.loc[sell_points, 'Portfolio_Value'], color='red', label='Sell', marker='v')
plt.title("Portfolio Evaluation with Transactions")
plt.legend()
plt.show()

In [ ]:
from flask import Flask, request, jsonify
from threading import Thread

app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    user_message = request.json.get('message', '')
    response = ""

    if "buy" in user_message.lower():
        response = "The strategy suggests buying when VIX is below its moving average."
    elif "sell" in user_message.lower():
        response = "The strategy suggests selling when the price is 10% above or 5% below the entry price."
    else:
        response = "I'm here to help! Ask me about buying, selling, or anomalies."
    
    return jsonify({'response': response})

@app.route('/')
def index():
    return '''
        <!DOCTYPE html>
        <html>
        <head>
          <title>Investment Strategy Chatbot</title>
          <link href="https://cdnjs.cloudflare.com/ajax/libs/bootstrap/5.3.0-alpha3/css/bootstrap.min.css" rel="stylesheet">
          <style>
           body {
              background-color: #f8f9fa;
           }
           .chat-container {
               max-width: 600px;
               margin: 50px auto;
               padding: 20px;
               background: white;
               border-radius: 8px;
               box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
           }
           .message {
              margin: 10px 0;
           }
           .bot-message {
              background: #d1e7dd;
              padding: 10px;
              border-radius: 8px;
           }
          .user-message {
              background: #f8d7da;
              padding: 10px;
              border-radius: 8px;
              text-align: right;
          }
          </style>
        </head>

    '''


def run_flask():
    app.run(debug=True, port=5000, use_reloader=False)

# Lancer le serveur Flask dans un thread parallèle
if __name__ == '__main__':
    flask_thread = Thread(target=run_flask)
    flask_thread.start()


In [ ]:
import requests

user_input = "What should I do about buying?"
response = requests.post("http://127.0.0.1:5000/chat", json={"message": user_input})
print(response.json())
